## Yellow Taxi Tripdata
### Daten verknüpfen und anreichern
Nachdem wir unseren Datensatz bereinigt haben, machen wir ihn jetzt verständlich. In dieser Phase verknüpfen wir die rohen numerischen Codes und IDs aus unserem Haupt-DataFrame mit den entsprechenden Informationen aus den Lookup-Tabellen.

Unser Hauptziel ist es, die numerischen LocationIDs und andere Codes durch beschreibende Namen zu ersetzen. So können wir später Analysen nicht nur mit Zahlen, sondern mit klaren Bezeichnungen wie "Manhattan" oder "Barzahlung" durchführen.

Wir werden die folgenden Schritte ausführen:

Die `PULocationID` und `DOLocationID` mit der `lookup_df-Tabelle` verknüpfen, um die Namen der Abhol- und Zielzonen hinzuzufügen.

Die Codes für `VendorID`, `RatecodeID` und `payment_type durch die entsprechenden Klartext-Namen ersetzen.

Am Ende dieser Phase haben wir einen Datensatz, der nicht nur sauber, sondern auch sofort lesbar und interpretierbar ist.


#### Pandas und SQL: Ein thematischer Vergleich

Die Verknüpfung von DataFrames in Pandas (pd.merge) ist genau das, was du in der Datenbankwelt mit einer JOIN-Anweisung in SQL tun würdest.

* In Pandas verwendest du pd.merge(), um zwei DataFrames zu kombinieren.

* In SQL verwendest du die JOIN-Anweisung, um zwei Tabellen in einer Datenbank zu verbinden.

Die Befehle funktionieren nach demselben Prinzip. Schauen wir uns zum Beispiel den Merge der LocationIDs an:

Pandas:

```Python

df.merge(lookup_df, left_on='PULocationID', right_on='LocationID', how='left')
```

SQL:

```SQL

SELECT ... FROM df
LEFT JOIN lookup_df ON df.PULocationID = lookup_df.LocationID
```

Wie du siehst, ist left_on und right_on das, was in SQL mit der ON-Klausel festgelegt wird. Und der how-Parameter in Pandas ('left') entspricht dem LEFT JOIN in SQL.

Beide Ansätze verfolgen das gleiche Ziel: Daten aus verschiedenen Quellen über eine gemeinsame Spalte zusammenzuführen.

### Hier ist der Plan für die nächsten Schritte:

* Lade die bereinigten Tripdaten: Du lädst dein new_york_taxi_cleaned.parquet-File in einen DataFrame (nennen wir ihn df).

* Lade die Lookup-Tabelle: Du lädst das taxi_zone_lookup.csv-File in einen anderen DataFrame (nennen wir ihn zonelookup).

* Merge für Abholorte: Wir verknüpfen die PULocationID im df mit der LocationID im zonelookup.

* Merge für Zielorte: Danach verknüpfen wir die DOLocationID im df mit der LocationID im zonelookup.

In [1]:
import pandas as pd
from fastparquet import ParquetFile

In [2]:
# Laden der Tripdaten:
pf = ParquetFile('NYTaxi-TripData/new_york_taxi_cleaned.parquet')
df = pf.to_pandas()

In [3]:
# Laden der lookup-Tabelle
zonelookup = pd.read_csv('NYTaxi-TripData/taxi+_zone_lookup.csv')

#### Hier noch ein Erklärung zur Zone-Lookup-Datei
Hier ist der Beginn der zonelookup:
- "LocationID","Borough","Zone","service_zone"
- 1,"EWR","Newark Airport","EWR"
- 2,"Queens","Jamaica Bay","Boro Zone"
- 3,"Bronx","Allerton/Pelham Gardens","Boro Zone"
...
Die Datei, die du da siehst, ist genau das, was wir brauchen! Das ist unsere Lookup-Tabelle für die Zonen, mit der wir die kryptischen LocationIDs aus dem Hauptdatensatz verständlich machen.

##### Hier ist, was die einzelnen Spalten bedeuten:

`LocationID`: Das ist der eindeutige Code für jede Taxizone. Diese ID ist der Schlüssel, den wir verwenden, um die Tabelle mit deinem großen DataFrame zu verknüpfen.

`Borough`: Der Bezirk (z. B. Queens, Bronx) in New York, zu dem die Zone gehört. Das ist sehr nützlich, um die Fahrten später nach Stadtteilen zu analysieren.

`Zone`: Der spezifische, sprechende Name der Taxizone (z. B. "Newark Airport" oder "Allerton/Pelham Gardens").

`service_zone`: Eine weitere Kategorie, die die Zonen nach dem Servicebereich gruppiert (z. B. "Boro Zone").

Diese kleine Datei ist der entscheidende Baustein, um deinen riesigen Datensatz mit den Taxifahrten zu einem verständlichen und aussagekräftigen Werkzeug für die Analyse zu machen.


In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2019-11-01 00:35:48,2019-11-01 01:13:12,1.0,8.5,1.0,N,138,161,1.0,32.0,3.0,0.5,8.35,6.12,0.3,50.27,2.5
1,1.0,2019-11-01 00:02:59,2019-11-01 00:05:26,1.0,0.3,1.0,N,239,142,2.0,3.5,3.0,0.5,0.00,0.00,0.3,7.30,2.5
2,1.0,2019-11-01 00:11:43,2019-11-01 00:19:43,1.0,1.9,1.0,N,142,262,2.0,8.5,3.0,0.5,0.00,0.00,0.3,12.30,2.5
3,1.0,2019-11-01 00:11:34,2019-11-01 00:18:02,1.0,0.5,1.0,N,246,186,2.0,5.5,3.0,0.5,0.00,0.00,0.3,9.30,2.5
4,1.0,2019-11-01 00:54:04,2019-11-01 01:05:24,2.0,2.4,1.0,N,246,239,1.0,10.0,3.0,0.5,2.75,0.00,0.3,16.55,2.5


In [5]:
#zonelookup.head()
zonelookup[235:240] # auf diese Weise können wir uns bestimmte Bereiche ansehen...!

,LocationID,Borough,Zone,service_zone
235,236,Manhattan,Upper East Side North,Yellow Zone
236,237,Manhattan,Upper East Side South,Yellow Zone
237,238,Manhattan,Upper West Side North,Yellow Zone
238,239,Manhattan,Upper West Side South,Yellow Zone
239,240,Bronx,Van Cortlandt Park,Boro Zone


>##### siehe oben
>in unseren tripdaten(df) haben wie die Spalten:
>* `PULocationID` (PickUp-Location)
>* `DOLocationID` (DropOff-Location)
>
>Diese Spalten werden nun verknüpft mit der Spalte `LocationID` aus dem Dataframe `zonelookup`.

In [6]:
# Führe den Merge für die Abholorte (PULocationID) aus
df = df.merge(zonelookup, left_on='PULocationID', right_on='LocationID', how='left')

# Benenne die neuen Spalten um, um Verwirrung zu vermeiden
df = df.rename(columns={'Borough': 'PU_Borough', 'Zone': 'PU_Zone'})

# Überprüfe die neuen Spalten, um sicherzustellen, dass alles funktioniert hat
print(df[['PULocationID', 'PU_Borough', 'PU_Zone']].head())

   PULocationID PU_Borough                    PU_Zone
0           138     Queens          LaGuardia Airport
1           239  Manhattan      Upper West Side South
2           142  Manhattan        Lincoln Square East
3           246  Manhattan  West Chelsea/Hudson Yards
4           246  Manhattan  West Chelsea/Hudson Yards


In [7]:
# Führe den Merge für die Zielorte (DOLocationID) aus
df = df.merge(zonelookup, left_on='DOLocationID', right_on='LocationID', how='left')

# Benenne die neuen Spalten um, um Verwirrung zu vermeiden
df = df.rename(columns={'Borough': 'DO_Borough', 'Zone': 'DO_Zone'})

# Zeige die neuen Spalten an, um sicherzustellen, dass alles funktioniert hat
print(df[['DOLocationID', 'DO_Borough', 'DO_Zone']].head())

   DOLocationID DO_Borough                       DO_Zone
0           161  Manhattan                Midtown Center
1           142  Manhattan           Lincoln Square East
2           262  Manhattan                Yorkville East
3           186  Manhattan  Penn Station/Madison Sq West
4           239  Manhattan         Upper West Side South


### SQL vs Pandas
#### ... und warum sollen die Spalten umbenannt werden?

Es ist verständlich, dass es verwirrend ist, wenn man die SQL-Logik im Kopf hat. Die Umbenennung ist tatsächlich ein großer Unterschied zwischen Pandas und SQL.

Lass es uns mit einer Analogie vergleichen:

**SQL-Logik**:
In SQL verwendest du Aliase (AS), um Spalten aus verschiedenen Tabellen auseinanderzuhalten.

SQL

``` sql
SELECT
    t1.LocationID AS PickupLocation,
    t2.LocationID AS DropoffLocation
FROM
    trips AS t1
LEFT JOIN
    zones AS t2 ON t1.PULocationID = t2.LocationID
```
Hier definierst du in einer einzigen SELECT-Anweisung, wie die Spalten in deiner neuen, kombinierten Tabelle heißen sollen. Es ist von Anfang an klar, welche ID zu welchem Zweck gehört.

---
**Pandas-Logik**:
In Pandas arbeitest du schrittweise. Du hast einen DataFrame und fügst ihm neue Spalten hinzu.

* Schritt 1: Du führst den ersten merge() aus, um die Abhol-Informationen hinzuzufügen. Dein DataFrame hat jetzt neue Spalten namens Borough und Zone.

* Schritt 2: Du möchtest jetzt die Ziel-Informationen hinzufügen. Wenn du jetzt wieder die zonelookup-Tabelle mergen würdest, versucht Pandas, wieder Spalten mit den Namen Borough und Zone hinzuzufügen. Pandas würde dich fragen, wie es mit den gleichnamigen Spalten umgehen soll.

>Die Umbenennung direkt nach dem ersten Merge ist wie das manuelle Festlegen eines Alias. Sie verhindert, dass es beim zweiten
>Merge zu einem Namenskonflikt kommt und stellt sicher, dass jede Spalte eindeutig dem Abhol- oder Zielort zugeordnet ist.
>
>Zusammenfassend:
>
>SQL macht alles in einem Schritt mit Aliasen.
>
>Pandas macht es schrittweise, daher müssen wir die Spaltennamen nach jedem Merge anpassen, um die Übersicht zu behalten.

Die Umbenennung ist also ein wichtiger Schritt, um deinen DataFrame konsistent und sauber zu halten, bevor wir den nächsten Merge durchführen.

In [8]:
df.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,total_amount,congestion_surcharge,LocationID_x,PU_Borough,PU_Zone,service_zone_x,LocationID_y,DO_Borough,DO_Zone,service_zone_y
0,1.0,2019-11-01 00:35:48,2019-11-01 01:13:12,1.0,8.5,1.0,N,138,161,1.0,...,50.27,2.5,138,Queens,LaGuardia Airport,Airports,161,Manhattan,Midtown Center,Yellow Zone
1,1.0,2019-11-01 00:02:59,2019-11-01 00:05:26,1.0,0.3,1.0,N,239,142,2.0,...,7.30,2.5,239,Manhattan,Upper West Side South,Yellow Zone,142,Manhattan,Lincoln Square East,Yellow Zone
2,1.0,2019-11-01 00:11:43,2019-11-01 00:19:43,1.0,1.9,1.0,N,142,262,2.0,...,12.30,2.5,142,Manhattan,Lincoln Square East,Yellow Zone,262,Manhattan,Yorkville East,Yellow Zone
3,1.0,2019-11-01 00:11:34,2019-11-01 00:18:02,1.0,0.5,1.0,N,246,186,2.0,...,9.30,2.5,246,Manhattan,West Chelsea/Hudson Yards,Yellow Zone,186,Manhattan,Penn Station/Madison Sq West,Yellow Zone
4,1.0,2019-11-01 00:54:04,2019-11-01 01:05:24,2.0,2.4,1.0,N,246,239,1.0,...,16.55,2.5,246,Manhattan,West Chelsea/Hudson Yards,Yellow Zone,239,Manhattan,Upper West Side South,Yellow Zone


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99574711 entries, 0 to 99574710
Data columns (total 26 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               float64       
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           float64       
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  LocationID_x        

### Endgültige Bereinigung
Da wir jetzt die neuen Spalten PU_Borough, PU_Zone, DO_Borough und DO_Zone haben, brauchen wir die ursprünglichen Spalten PULocationID und DOLocationID nicht mehr. Sie sind jetzt redundant.

Wir entfernen auch die überschüssigen Spalten, die beim Merge hinzugefügt wurden (LocationID, Borough, Zone, service_zone), da wir sie bereits umbenannt haben.

Führe diesen Code aus, um den DataFrame aufzuräumen:

In [10]:
df = df.drop(columns=['PULocationID', 'DOLocationID', 'LocationID', 'Borough', 'Zone', 'service_zone'], errors='ignore')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99574711 entries, 0 to 99574710
Data columns (total 24 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               float64       
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   payment_type           float64       
 8   fare_amount            float64       
 9   extra                  float64       
 10  mta_tax                float64       
 11  tip_amount             float64       
 12  tolls_amount           float64       
 13  improvement_surcharge  float64       
 14  total_amount           float64       
 15  congestion_surcharge   float64       
 16  LocationID_x           int64         
 17  PU_Borough             object        
 18  PU_Zone             

In [13]:
df.to_parquet('NYTaxi-TripData/new_york_taxi_cleaned_final.parquet', index=False)